In [1]:
import igraph as ig
import numpy as np
import os
import pandas as pd
import random
from time import time
import math
import powerlaw
import matplotlib.pyplot as plt
import networkx as nx
import nx_cugraph as nxcg
from auxiliar_bb import noise_corrected, disparity
from auxiliar_projections_large import apply_projection

In [2]:
FILENAME = "/home/daniel/Documents/phd/phd-thesis-lab/12-third_year/00-Data/05-actor-movie/actor-movie.graphml"
PROJ_NAME = ["simple", "jaccard", "master", "hyper", "resall"]

In [3]:
###### ****** Read BI GRAPH ****** ######
g = ig.read(FILENAME)
print(g.summary())
print()

user_nodes = g.vs.select(type=0)
res_nodes = g.vs.select(type=1)

if(g.is_bipartite()): # Check if the the graph is bipartite
    print("The graph IS bipartite")
else:
    print("The graph IS NOT bipartite")
    exit()
print("|U|=",len(user_nodes), " \t|R|=",len(res_nodes), " \t|U|+|R|=",
      len(user_nodes)+len(res_nodes), "=", g.vcount())
print()
###### ****** END ****** ######

IGRAPH UN-T 511463 1470404 -- 
+ attr: id (v), name (v), type (v)

The graph IS bipartite
|U|= 383640  	|R|= 127823  	|U|+|R|= 511463 = 511463



In [10]:
def escalar_pesos(grafo):
    """Función para escalar los pesos."""
    ### Factor de Escalado
    edges_temp = grafo.es["weight"]
    print(f"Peso máximo={max(edges_temp)} y mínimo={min(edges_temp)} en aristas: ")
    print()

    ### Determinar si el peso minimo es 0
    if min(edges_temp) == 0.0:
        # Se obtiene el minimu segundo
        min_val = sorted(set(grafo.es["weight"]))[1]
        factor_escala = math.ceil(1 / min_val)
        print("Factor de escala:", factor_escala)

    else:
        factor_escala = math.ceil(1 / min(edges_temp))
        print("Factor de escala:", factor_escala)
    
    grafo.es["weight"] = (np.array(edges_temp) * factor_escala).round().astype(int)
    for edge in grafo.es:
        if edge["weight"] == 0:
            edge["weight"] = 1
    
    return grafo

In [5]:
for proj_opcion in PROJ_NAME:
    ###### ****** Projections ****** ######
    user_graph = apply_projection(g, proj_opcion,
                                len(user_nodes), False) # False = Users = 0
    print("Done PROJ1 - Users Projection")
    edges_temp = user_graph.es()["weight"]
    print(f"Peso máximo={max(edges_temp)} y mínimo={min(edges_temp)} en aristas: ")

    rsrs_graph = apply_projection(g, proj_opcion,
                                len(user_nodes), True) # True = Resources = 1
    print("\nDone PROJ2 - Resources Projection")
    edges_temp = rsrs_graph.es()["weight"]
    print(f"Peso máximo={max(edges_temp)} y mínimo={min(edges_temp)} en aristas: ")
    print()

    ###### ****** BACKBONING RESOURCES ****** ######
    g_toy = user_graph.copy() # Graph to analyze
    print("\n##### **** BACKBONING USERS **** #####")
    print("Projection Name:", proj_opcion)
    print("Summary\n",g_toy.summary())
    print("##### END #####")
    print()

    g_toy = escalar_pesos(g_toy)
    print(f"Peso máximo={max(g_toy.es['weight'])} y mínimo={min(g_toy.es['weight'])} en aristas: ")
    print()

    ### Disparity filter ###
    a = time()
    bb_df = disparity(g_toy)
    b = time() - a
    print("BOT DF - time: %.10f seconds." % b)
    for alpha__, g__ in bb_df.items():
        print(f"Grafo filtrado con alpha={alpha__}: {g__.summary()}")
        flname = (
            "am/top/amz_top_" + proj_opcion + "_DF_alpha" + str(alpha__)[2:] + ".graphml"
        )
        g__.write_graphml(flname)
    print("================================")

    # Noise Corrected
    a = time()
    bb_nc = noise_corrected(g_toy)
    b = time() - a
    print("BOT NC - time: %.10f seconds." % b)
    for alpha__, g__ in bb_nc.items():
        print(f"Grafo filtrado con alpha={alpha__}: {g__.summary()}")
        flname = (
            "am/top/amz_top_" + proj_opcion + "_NC_alpha" + str(alpha__)[2:] + ".graphml"
        )
        g__.write_graphml(flname)
    print("================================")
    print()
    print("##### ***** Done RSCS ***** #####")
    ###### ****** END ****** ######



    g_toy = rsrs_graph.copy() # Graph to analyze
    print("\n##### **** BACKBONING USERS **** #####")
    print("Projection Name:", proj_opcion)
    print("Summary\n",g_toy.summary())
    print("##### END #####")
    print()

    g_toy = escalar_pesos(g_toy)
    print(f"Peso máximo={max(g_toy.es['weight'])} y mínimo={min(g_toy.es['weight'])} en aristas: ")
    print()

    ### Disparity filter ###
    a = time()
    bb_df = disparity(g_toy)
    b = time() - a
    print("BOT DF - time: %.10f seconds." % b)
    for alpha__, g__ in bb_df.items():
        print(f"Grafo filtrado con alpha={alpha__}: {g__.summary()}")
        flname = (
            "am/bot/amz_bot_" + proj_opcion + "_DF_alpha" + str(alpha__)[2:] + ".graphml"
        )
        g__.write_graphml(flname)
    print("================================")

    # Noise Corrected
    a = time()
    bb_nc = noise_corrected(g_toy)
    b = time() - a
    print("BOT NC - time: %.10f seconds." % b)
    for alpha__, g__ in bb_nc.items():
        print(f"Grafo filtrado con alpha={alpha__}: {g__.summary()}")
        flname = (
            "am/bot/amz_bot_" + proj_opcion + "_NC_alpha" + str(alpha__)[2:] + ".graphml"
        )
        g__.write_graphml(flname)
    print("================================")
    print()
    print("##### ***** Done RSCS ***** #####")
    ###### ****** END ****** ######

    ###### ****** END ****** ######

KeyboardInterrupt: 

In [6]:
###### ****** BACKBONING RESOURCES ****** ######
g_toy = rsrs_graph.copy() # Graph to analyze
print("\n##### **** BACKBONING USERS **** #####")
print("Projection Name:", PROJ_NAME)
print("Summary\n",g_toy.summary())
print("##### END #####
print()

g_toy = escalar_pesos(g_toy)
print(f"Peso máximo={max(g_toy.es['weight'])} y mínimo={min(g_toy.es['weight'])} en aristas: ")
print()

NameError: name 'rsrs_graph' is not defined

In [5]:
def bipartite_cc_uu_prime(graph, u_id, u_prime_id):
    """
    Calculates the Jaccard index based clustering coefficient for a pair of vertices
    u and u' from the same set of nodes in a bipartite graph.

    Args:
        graph: An igraph Graph object. Must be bipartite with a 'type' vertex attribute.
        u_id: The ID of the first vertex.
        u_prime_id: The ID of the second vertex.

    Returns:
        The Jaccard index (cc_u_u_prime) or 0 if union of neighbors is empty.
    """
    if not graph.is_bipartite():
        raise ValueError("Graph must be bipartite.")

    # Get neighbors of u and u'
    neighbors_u = set(graph.neighbors(u_id))
    neighbors_u_prime = set(graph.neighbors(u_prime_id))

    # Calculate intersection and union
    intersection = len(neighbors_u.intersection(neighbors_u_prime))
    union = len(neighbors_u.union(neighbors_u_prime))

    if union == 0:
        return 0.0
    return intersection / union
 
def local_bipartite_clustering_coefficient(graph, u_id, U_type_value=False):
    """
    Calculates the local clustering coefficient for a vertex u in a bipartite graph.
    The formula uses neighbors of neighbors (N(N(u))) that are of the same type as u.

    Args:
        graph: An igraph Graph object. Must be bipartite with a 'type' vertex attribute
               (e.g., True/False or 0/1 for the two partitions).
        u_id: The ID of the vertex for which to calculate the local clustering coefficient.
        U_type_value: The boolean value (True/False) that indicates the partition
                      to which node u belongs. All nodes in U should have this type.

    Returns:
        The local clustering coefficient for vertex u, or 0 if N(N(u)) is empty.
    """
    if not graph.is_bipartite():
        raise ValueError("Graph must be bipartite.")
    if "type" not in graph.vs.attributes():
        raise ValueError("Bipartite graph must have a 'type' vertex attribute.")

    # Ensure u_id is of the specified U_type_value
    if graph.vs[u_id]["type"] != U_type_value:
        raise ValueError(f"Vertex {u_id} does not belong to the specified partition U.")

    # Get neighbors of u
    neighbors_u = graph.neighbors(u_id)
    
    # Get neighbors of neighbors of u, filtering for nodes of the same type as u
    # These are the u' nodes in N(N(u)) that are in the same partition U
    nn_u = set()
    for v_neighbor_id in neighbors_u:
        for nn_id in graph.neighbors(v_neighbor_id):
            if graph.vs[nn_id]["type"] == U_type_value and nn_id != u_id: # Exclude u itself
                nn_u.add(nn_id)

    if not nn_u:
        return 0.0

    sum_cc_uu_prime = 0.0
    for u_prime_id in nn_u:
        sum_cc_uu_prime += bipartite_cc_uu_prime(graph, u_id, u_prime_id)

    return sum_cc_uu_prime / len(nn_u)

def average_local_bipartite_clustering_coefficient(graph, U_type_value=False):
    """
    Calculates the average local clustering coefficient for a set of nodes U
    in a bipartite graph.

    Args:
        graph: An igraph Graph object. Must be bipartite with a 'type' vertex attribute.
        U_type_value: The boolean value (True/False) that indicates the partition
                      for which to calculate the average clustering coefficient.

    Returns:
        The average local clustering coefficient for the set U.
    """
    if not graph.is_bipartite():
        raise ValueError("Graph must be bipartite.")
    if "type" not in graph.vs.attributes():
        raise ValueError("Bipartite graph must have a 'type' vertex attribute.")

    # Get all vertices in set U
    U_vertices_ids = [v.index for v in graph.vs if v["type"] == U_type_value]

    if not U_vertices_ids:
        return 0.0

    sum_cc_u = 0.0
    for u_id in U_vertices_ids:
        sum_cc_u += local_bipartite_clustering_coefficient(graph, u_id, U_type_value)

    return sum_cc_u / len(U_vertices_ids)

def compute_power_law_bipartite(gb, type_n=1):
    """Calcula el alpha del bipartita"""
    fit = powerlaw.Fit(gb.degree(gb.vs.select(type=type_n)), discrete=True, verbose=False)
    return fit.alpha

def compute_power_law(g):
    """Calcula el alpha del proyectado"""
    fit = powerlaw.Fit(g.degree(), discrete=True, verbose=False)
    return fit.alpha

In [38]:
def compute_avg_path_length(g, k):
    G = g.to_networkx()
    nxG = nxcg.from_networkx(G)
    all_nodes = list(G.nodes())
    sample_nodes = random.sample(all_nodes, k)
    total_distances = 0
    reachable_pairs = 0
    for i, src_node in enumerate(sample_nodes):
        sssp_df = nxcg.shortest_path_length(nxG, source=src_node)
        total_distances += sum(sssp_df.values())
        reachable_pairs += len(sssp_df.values())

    apl_approx = float(total_distances) / reachable_pairs
    return apl_approx


def compute_bip_metrics(gb, typen):
    """Calcula x1,x2,x3,x8,x9,γ_Ub del grafo bipartito."""
    x1 = len(gb.vs.select(type=0))
    x2 = len(gb.vs.select(type=1))
    x3 = gb.ecount()
    #x8 = average_local_bipartite_clustering_coefficient(gb, U_type_value=typen)    
    x8 = 1
    x9 = compute_avg_path_length(gb, 100)
    #try:
    #    x9 = compute_avg_path_length(gb, 100)
    #except:
    #    x9 = np.inf
    x11 = compute_power_law_bipartite(gb, typen)
    return dict(x1=x1, x2=x2, x3=x3, x8=x8, x9=x9, x11=x11)

def compute_proj_metrics(gu):
    """Calcula x4,x5,x6,x7,x10,γ_U de la proyección."""
    x4 = gu.vcount()
    x5 = gu.ecount()
    x6 = len(gu.clusters(mode='weak'))
    x7 = gu.transitivity_undirected(mode="zero")
    x10 = compute_avg_path_length(gu, 100)
    #try:
    #    x10 = gu.average_path_length(directed=False)
    #except:
    #    x10 = np.inf
    x12 = compute_power_law(gu)
    return dict(x4=x4, x5=x5, x6=x6, x7=x7, x10=x10, x12=x12)

def evaluate_solution(bip, proj, typen):
    """Dado bip y proj metrics, arma x, f, g."""
    # unimos diccionarios
    x = {
        **bip,
        **proj
    }
    # objetivos
    f = np.array([
        abs(x["x1"] - x["x4"]) if typen==0 else abs(x["x2"] - x["x4"]),
        (2*x["x5"]) / (x["x4"]*(x["x4"]-1)) if x["x4"]>1 else np.inf,
        x["x6"],
        1 - x["x7"],
        abs(x["x11"] - x["x12"]),
        abs(x["x9"] - x["x10"]),
    ])
    # restricciones g_i(x)<=0
    #g = np.array([
    #    f[1] - x["x3"]/(x["x1"]*x["x2"]) if x["x1"]*x["x2"]>0 else np.inf,
    #])
    #return dict(metrics=x, f=f, g=g, graph=proj)
    return dict(metrics=x, f=f, graph=proj)

def is_feasible(sol):
    return np.all(sol["g"] <= 0)

def pareto_front(sols):
    front = []
    for i, si in enumerate(sols):
        if any(np.all(sj["f"] <= si["f"]) and np.any(sj["f"] < si["f"])
               for j, sj in enumerate(sols) if i!=j):
            continue
        front.append(si)
    return front

def crowding_distance(front):
    N, k = len(front), front[0]["f"].size
    F = np.array([s["f"] for s in front])
    dist = np.zeros(N)
    for m in range(k):
        idx = np.argsort(F[:,m])
        f_min, f_max = F[idx[0],m], F[idx[-1],m]
        dist[idx[0]] = dist[idx[-1]] = np.inf
        if f_max == f_min: continue
        for i in range(1, N-1):
            dist[idx[i]] += (F[idx[i+1],m] - F[idx[i-1],m]) / (f_max - f_min)
    return dist

def pareto_rank_all(solutions):
    """
    Clasifica todas las soluciones en frentes de Pareto.
    Devuelve una lista de listas: cada sublista contiene un frente.
    """
    remaining = solutions.copy()
    fronts = []
    
    while remaining:
        current_front = []
        for i, si in enumerate(remaining):
            dominated = False
            for j, sj in enumerate(remaining):
                if i == j:
                    continue
                if np.all(sj["f"] <= si["f"]) and np.any(sj["f"] < si["f"]):
                    dominated = True
                    break
            if not dominated:
                current_front.append(si)
        
        fronts.append(current_front)
        remaining = [s for s in remaining if all(s is not r for r in current_front)]
    
    return fronts



In [39]:
# —————————— Flujo principal ——————————
tnodes = 1

# 1) Leer el único grafo bipartito
gb = ig.Graph.Read_GraphML(FILENAME)
bip_metrics = compute_bip_metrics(gb, tnodes)
bip_metrics

{'x1': 383640,
 'x2': 127823,
 'x3': 1470404,
 'x8': 1,
 'x9': 7.33979227379005,
 'x11': 5.066327861107771}

In [40]:
# 2) Escanear carpeta de proyecciones
if tnodes == 0:
    proj_dir = "am/top"    
else:
    proj_dir = "am/bot"
    
proj_files = [f for f in os.listdir(proj_dir)
              if f.endswith(".graphml") and f!="bipartito.graphml"]


# 3) Calcular soluciones
solutions = []
conta_neg = 0
for fname in proj_files:
    gu = ig.Graph.Read_GraphML(os.path.join(proj_dir, fname))
    proj_metrics = compute_proj_metrics(gu)
    sol = evaluate_solution(bip_metrics, proj_metrics, tnodes)
    sol["filename"] = fname  # <- Añadimos esta línea
    solutions.append(sol)
    #if is_feasible(sol):
    #    solutions.append(sol)
    
print("Soluciones factibles:", len(solutions))

Soluciones factibles: 40


In [41]:
to_save = pd.DataFrame(solutions)
if tnodes == 0:
    to_save.to_csv("datos_proj_top_act_mov.csv")
else:
    to_save.to_csv("datos_proj_bot_act_mov.csv")


In [43]:
all_fronts = pareto_rank_all(solutions)

for rank, front in enumerate(all_fronts, 1):
    print(f"\nFrente {rank} ({len(front)} soluciones):")
    for sol in front:
        print(f" - {sol['filename']} — f = {sol['f']}")



Frente 1 (38 soluciones):
 - amz_bot_resall_DF_alpha2.graphml — f = [0.00000000e+00 4.90684035e-04 3.69200000e+03 8.72838335e-01
 4.83071582e-01 3.31086491e+00]
 - amz_bot_simple_DF_alpha2.graphml — f = [0.00000000e+00 1.95221846e-04 3.20160000e+04 8.33028358e-01
 7.69211945e-01 1.89457072e+00]
 - amz_bot_resall_DF_alpha15.graphml — f = [0.00000000e+00 3.80479676e-04 3.84700000e+03 8.70840749e-01
 4.68507632e-01 3.14995272e+00]
 - amz_bot_resall_NC_alpha1.graphml — f = [0.00000000e+00 2.50228436e-03 3.14200000e+03 7.84836383e-01
 7.49878687e-01 4.17802625e+00]
 - amz_bot_simple_NC_alpha15.graphml — f = [0.00000000e+00 2.50365474e-03 3.14200000e+03 7.84668346e-01
 7.86315936e-01 4.25678189e+00]
 - amz_bot_master_NC_alpha2.graphml — f = [0.00000000e+00 1.69059934e-03 3.14300000e+03 8.31412509e-01
 6.21936873e-01 4.01565772e+00]
 - amz_bot_jaccard_NC_alpha2.graphml — f = [0.00000000e+00 2.50388168e-03 3.14200000e+03 7.83944974e-01
 8.16354934e-01 4.13137561e+00]
 - amz_bot_jaccard_DF_alp

In [35]:

cd = crowding_distance(all_fronts[0])
pareto_sorted = [s for _, s in sorted(zip(-cd, all_fronts[0]), key=lambda x: x[0])]
print("Crowding", len(pareto_sorted))
print(pareto_sorted)

Crowding 39
[{'metrics': {'x1': 383640, 'x2': 127823, 'x3': 1470404, 'x8': 0.25, 'x9': 7.086370967654382, 'x11': 1.8623650345000595, 'x4': 383640, 'x5': 13876203, 'x6': 3341, 'x7': 0.14419478043132772, 'x10': 3.7173506969406285, 'x12': 2.035315467847756}, 'f': array([0.00000000e+00, 1.88561927e-04, 3.34100000e+03, 8.55805220e-01,
       1.72950433e-01, 3.36902027e+00]), 'graph': {'x4': 383640, 'x5': 13876203, 'x6': 3341, 'x7': 0.14419478043132772, 'x10': 3.7173506969406285, 'x12': 2.035315467847756}, 'filename': 'amz_top_master_NC_alpha1.graphml'}, {'metrics': {'x1': 383640, 'x2': 127823, 'x3': 1470404, 'x8': 0.25, 'x9': 7.086370967654382, 'x11': 1.8623650345000595, 'x4': 383639, 'x5': 866733, 'x6': 140175, 'x7': 0.7973348651930043, 'x10': 12.925247141828782, 'x12': 4.052279282334988}, 'f': array([1.00000000e+00, 1.17779840e-05, 1.40175000e+05, 2.02665135e-01,
       2.18991425e+00, 5.83887617e+00]), 'graph': {'x4': 383639, 'x5': 866733, 'x6': 140175, 'x7': 0.7973348651930043, 'x10': 1

In [15]:

# Asumiendo que ya tienes: all_fronts = pareto_rank_all(solutions)

# Preparar datos
points = []
labels = []

for front_idx, front in enumerate(all_fronts):
    for sol in front:
        points.append(sol["f"])
        labels.append(front_idx)  # índice de frente

points = np.array(points)
labels = np.array(labels)

# Aplicar t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
embedding = tsne.fit_transform(points)

# Graficar
plt.figure(figsize=(8, 6))
scatter = plt.scatter(embedding[:, 0], embedding[:, 1],
                      c=labels, cmap="tab10", s=50, edgecolors='k')

plt.title("Visualización t-SNE de soluciones por frente de Pareto")
plt.xlabel("t-SNE componente 1")
plt.ylabel("t-SNE componente 2")
plt.grid(True, linestyle="--", alpha=0.3)
legend = plt.legend(*scatter.legend_elements(), title="Frente", loc="upper right")
plt.tight_layout()
plt.show()


NameError: name 'TSNE' is not defined